In [46]:
# Kristina (temp code)
# crime-air quality correlation 

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.dates as mdates

In [13]:
df_sf_crime = pd.read_csv('sf-crime-results-20181007-143916.csv')
df_sf_crime['timestamp'] = pd.to_datetime(df_sf_crime['timestamp'], format="%Y%m%dT%H:%M:%S.%f")
df_sf_crime_subset= df_sf_crime[['timestamp', 'location', 'category']]
# need to get counts per capita
df_sf_crime_subset = df_sf_crime_subset.groupby([df_sf_crime_subset['timestamp'].dt.year.rename('year'), df_sf_crime_subset['timestamp'].dt.month.rename('month')]).size().reset_index(name='Count')
df_sf_crime_subset.head()

,year,month,Count
0,2003,1,56
1,2003,2,49
2,2003,3,83
3,2003,4,76
4,2003,5,63


In [37]:
df_CO_pollution = pd.read_csv('carbon-monoxide-results-20181007-145932.csv')
df_CO_pollution['date_local'] = pd.to_datetime(df_CO_pollution['date_local'], format="%Y-%m-%d")
df_CO_pollution['location'] = df_CO_pollution['latitude'].astype(str) +','+  df_CO_pollution["longitude"].astype(str)
df_CO_pollution_subset = df_CO_pollution[['date_local', 'arithmetic_mean']]
df_CO_pollution_subset = df_CO_pollution_subset.groupby([df_CO_pollution_subset['date_local'].dt.year.rename('year'), df_CO_pollution_subset['date_local'].dt.month.rename('month')]).mean().reset_index()

In [40]:
df_merged = pd.merge(df_CO_pollution_subset, df_sf_crime_subset, on=['year', 'month'], left_index=True)
df_merged.head()

,year,month,arithmetic_mean,Count
0,2003,1,0.865485,56
1,2003,2,0.760733,49
2,2003,3,0.640547,83
3,2003,4,0.649492,76
4,2003,5,0.570179,63


In [42]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_merged['arithmetic_mean'], df_merged['Count'].values, test_size=0.33, random_state=42)

from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(max_depth=2, random_state=0, n_estimators=100)
regr.fit(X_train.as_matrix().reshape(-1,1), y_train)

/home/kristina/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=2,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [43]:
y_pred = regr.predict(X_test.as_matrix().reshape(-1,1))

/home/kristina/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [44]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred)

847.4215479972057

In [45]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

-0.10540494604535322